# pymysql 실습

- pip install pymysql

In [1]:
import pymysql

# 1. DB연결
- id, pw, host, port, db

In [2]:
# '127.0.0.1' = 'localhost'
conn = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'YuJeong',
    password = '1234',
    db = 'csm_db',
    charset = 'utf8'
)

# 2. DB cursor 생성
- 데이터 셔틀버스와 같음

In [ ]:
# 커서 객체 생성
cur = conn.cursor()

In [23]:
# items 테이블이 존재하면 삭제
cur.execute("drop table if exists items")

0

# 3. sql 명령 실행

In [28]:
# itmes 테이블 만들기 : 테이블이 존재하지 않으면 생성
create_sql = """
CREATE TABLE IF NOT EXISTS items(
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    code TEXT NOT NULL,
    name TEXT NOT NULL,
    price INTEGER NULL
)
"""
cur.execute(create_sql)

0

## 데이터 입력 (Create)

In [29]:
# 레코드 1개 테이블에 추가하기
INSERT_SQL= """
INSERT INTO items(code, name, price)
    VALUES(%s, %s, %s)
"""
cur.execute(INSERT_SQL, ('A001', 'TV', 1000000))

1

In [30]:
# 변경사항 저장, 반드시 실행해야만 테이블에 반영됨.
conn.commit()

In [31]:
# 여러개의 레코드를 한꺼번에 추가하기
datas = (
    ('A0002', '에어컨 30평형', 2000000),
    ('A0003', '최신형 스마트폰', 1000000),
    ('A0004', '최신형 노트북', 500000)
)
cur.executemany(INSERT_SQL, datas)

3

In [32]:
# 변경사항 저장, 반드시 실행해야만 테이블에 반영됨.
conn.commit()

## 데이터 읽기 (Read)

In [33]:
# select 문
select_sql1 = "SELECT * FROM items"
cur.execute(select_sql1)

4

In [ ]:
# DB에서 조회한 결과를 확인하기 위해서는 fetch 수행
rows = cur.fetchall()

In [18]:
rows

((1, 'A001', 'TV', 1000000),
 (2, 'A0002', '에어컨 30평형', 2000000),
 (3, 'A0003', '최신형 스마트폰', 1000000),
 (4, 'A0004', '최신형 노트북', 500000))

In [40]:
for idx, row in enumerate(rows):
    print(f"{idx+1} - {row[1:]}")

1 - ('A001', 'TV', 1000000)
2 - ('A0002', '에어컨 30평형', 2000000)
3 - ('A0003', '최신형 스마트폰', 1000000)
4 - ('A0004', '최신형 노트북', 500000)


In [41]:
# select 문, 2개의 레코드
select_sql1 = "SELECT * FROM items LIMIT 3"
cur.execute(select_sql1)

3

In [42]:
rows = cur.fetchall()
rows

((1, 'A001', 'TV', 1000000),
 (2, 'A0002', '에어컨 30평형', 2000000),
 (3, 'A0003', '최신형 스마트폰', 1000000))

In [43]:
# fetchall 한번 수행하면 데이터는 비워짐
# 위에서 fetchall() 실행했기 때문에 빈결과가 나옴
cur.fetchall()

()

## 수정 (Update)

In [44]:
update_sql = """
UPDATE items
SET price = %s
WHERE code = %s
"""

price = 1500000
code = 'A0001'
cur.execute(update_sql, (price, code))

1

In [45]:
conn.commit()

In [48]:
# id = 2인 레코드의 price -> 2500000 변경
update_sql = """
UPDATE items
SET price = %s
WHERE id = %s
"""

price = 2500000
code = 2
cur.execute(update_sql, (price, code))
conn.commit()

## 데이터 삭제 (Delete)

In [58]:
delete_sql = """
DELETE FROM items
WHERE code = %s

"""
cur.execute(delete_sql, 'A0003') # %s 1개일 경우, tuple 생략 가능
conn.commit()

# 4. commit, rollback 실행
- insert, update, delete sql을 실행했을 경우는 반드시 실행해야 DB에 반영됨

In [ ]:
# 데이터 생성하기
# code A0005, name 로봇청소기, price 1000000
INSERT_SQL = """
INSERT INTO items(code, name, price)
    VALUES(%s, %s, %s)
"""
cur.execute(INSERT_SQL, ('A0005', '로봇청소기', 1000000))
conn.commit()

In [62]:
INSERT_SQL = """
INSERT INTO items(code, name, price)
    VALUES(%s, %s, %s)
"""
cur.execute(INSERT_SQL, ('A0006', '스타일러', 3500000))

1

In [63]:
conn.rollback()

In [ ]:
# DB에 가장 마지막에 반영한 이후 시점까지 되돌리기
conn.rollback()

In [64]:
# 조회
select_sql1 = "SELECT * FROM items"
cur.execute(select_sql1)
rows = cur.fetchall()
rows

((1, 'A0001', 'TV', 1500000),
 (2, 'A0002', '에어컨 30평형', 2500000),
 (4, 'A0004', '최신형 노트북', 500000),
 (5, 'A0005', '로봇청소기', 1000000))

# 5. DB close()
- 데이터 처리를 위해 연결한 DB연결 자원을 반환함

In [ ]:
conn.close()